In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('Groceries_dataset.csv')
df

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk
...,...,...,...
38760,4471,08-10-2014,sliced cheese
38761,2022,23-02-2014,candy
38762,1097,16-04-2014,cake bar
38763,1510,03-12-2014,fruit/vegetable juice


In [4]:
df.isnull().sum()

Member_number      0
Date               0
itemDescription    0
dtype: int64

In [5]:
df['itemDescription'].value_counts().head()

itemDescription
whole milk          2502
other vegetables    1898
rolls/buns          1716
soda                1514
yogurt              1334
Name: count, dtype: int64

In [6]:
member_shopping_frequency = df.groupby('Member_number')['Date'].count().sort_values(ascending=False)
print(member_shopping_frequency)

Member_number
3180    36
3737    33
3050    33
2051    33
3915    31
        ..
3533     2
2302     2
4824     2
1084     2
3377     2
Name: Date, Length: 3898, dtype: int64


In [7]:
from apyori import apriori

In [8]:
group_same_customer = df.sort_values(by = 'Member_number',ascending = True)
group_same_customer['itemDescription'].str.strip()

1629                    soda
13331             whole milk
8395              whole milk
4843                 sausage
17778     pickled vegetables
                ...         
34885    semi-finished bread
25489       other vegetables
9340            bottled beer
27877                 onions
3578                    soda
Name: itemDescription, Length: 38765, dtype: object

In [9]:
transactions_list = []
for _, group in group_same_customer.groupby('Member_number'):
    customer_purchases = group['itemDescription'].tolist()
    transactions_list.append(customer_purchases)
transactions_list

[['soda',
  'whole milk',
  'whole milk',
  'sausage',
  'pickled vegetables',
  'canned beer',
  'yogurt',
  'misc. beverages',
  'salty snack',
  'sausage',
  'semi-finished bread',
  'hygiene articles',
  'pastry'],
 ['whole milk',
  'sausage',
  'curd',
  'soda',
  'beef',
  'frankfurter',
  'white bread',
  'frankfurter',
  'whipped/sour cream',
  'rolls/buns',
  'whole milk',
  'soda'],
 ['other vegetables',
  'frozen vegetables',
  'specialty chocolate',
  'sugar',
  'butter',
  'whole milk',
  'tropical fruit',
  'butter milk'],
 ['rolls/buns',
  'dental care',
  'detergent',
  'sausage',
  'rolls/buns',
  'rolls/buns',
  'frozen meals',
  'root vegetables'],
 ['whole milk',
  'packaged fruit/vegetables',
  'whole milk',
  'canned beer',
  'chocolate',
  'dish cleaner',
  'shopping bags',
  'other vegetables',
  'pip fruit',
  'red/blush wine',
  'root vegetables',
  'whole milk',
  'frozen fish',
  'hygiene articles',
  'pastry',
  'chocolate',
  'rolls/buns',
  'other vegetab

In [28]:
association_rules = apriori(transactions_list, min_support=0.001, min_confidence=0.05, min_lift=2, min_length=2,max_length = 2)
rules_list = list(association_rules)
rules_list

[RelationRecord(items=frozenset({'Instant food products', 'cake bar'}), support=0.001026167265264238, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Instant food products'}), items_add=frozenset({'cake bar'}), confidence=0.06666666666666667, lift=2.9198501872659173)]),
 RelationRecord(items=frozenset({'canned fish', 'Instant food products'}), support=0.0012827090815802976, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Instant food products'}), items_add=frozenset({'canned fish'}), confidence=0.08333333333333333, lift=2.82463768115942)]),
 RelationRecord(items=frozenset({'Instant food products', 'chocolate'}), support=0.0028219599794766545, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Instant food products'}), items_add=frozenset({'chocolate'}), confidence=0.18333333333333332, lift=2.1205736894164193)]),
 RelationRecord(items=frozenset({'condensed milk', 'Instant food products'}), support=0.001026167265264238, ordered_statistics=[OrderedStatisti

In [29]:
len(rules_list)

406

In [30]:
def print_rules(rules):
    for rule in rules:
        print ('rule.items=', list(rule.items))
        print ('rule.support=',rule.support)

        for os in rule.ordered_statistics:
            print ('\titems_base=', list(os.items_base))
            print ('\tlifted_item =', list(os.items_add))
            print ('\tlift=', os.lift)
            print ('\tconfidence (i.e. cond prob {} if {})='.format(list(os.items_add), list(os.items_base)), os.confidence)
            print ('\n')
        
print_rules(rules_list)

rule.items= ['Instant food products', 'cake bar']
rule.support= 0.001026167265264238
	items_base= ['Instant food products']
	lifted_item = ['cake bar']
	lift= 2.9198501872659173
	confidence (i.e. cond prob ['cake bar'] if ['Instant food products'])= 0.06666666666666667


rule.items= ['canned fish', 'Instant food products']
rule.support= 0.0012827090815802976
	items_base= ['Instant food products']
	lifted_item = ['canned fish']
	lift= 2.82463768115942
	confidence (i.e. cond prob ['canned fish'] if ['Instant food products'])= 0.08333333333333333


rule.items= ['Instant food products', 'chocolate']
rule.support= 0.0028219599794766545
	items_base= ['Instant food products']
	lifted_item = ['chocolate']
	lift= 2.1205736894164193
	confidence (i.e. cond prob ['chocolate'] if ['Instant food products'])= 0.18333333333333332


rule.items= ['condensed milk', 'Instant food products']
rule.support= 0.001026167265264238
	items_base= ['Instant food products']
	lifted_item = ['condensed milk']
	lift= 2

In [39]:

list_item = []
list_lifted_item = []
list_lift = []
for rule in rules_list:
    for os in rule.ordered_statistics:
        list_item.append(list(os.items_base))
        list_lifted_item.append(list(os.items_add))
        list_lift.append(os.lift)
        
list_item

[['Instant food products'],
 ['Instant food products'],
 ['Instant food products'],
 ['Instant food products'],
 ['Instant food products'],
 ['flower (seeds)'],
 ['Instant food products'],
 ['Instant food products'],
 ['Instant food products'],
 ['Instant food products'],
 ['Instant food products'],
 ['soups'],
 ['house keeping products'],
 ['kitchen towels'],
 ['UHT-milk'],
 ['liquor'],
 ['rice'],
 ['tea'],
 ['abrasive cleaner'],
 ['artif. sweetener'],
 ['artif. sweetener'],
 ['artif. sweetener'],
 ['artif. sweetener'],
 ['artif. sweetener'],
 ['artif. sweetener'],
 ['artif. sweetener'],
 ['finished products'],
 ['baking powder'],
 ['frozen fish'],
 ['mustard'],
 ['popcorn'],
 ['baking powder'],
 ['pot plants'],
 ['specialty cheese'],
 ['zwieback'],
 ['bathroom cleaner'],
 ['cocoa drinks'],
 ['flower soil/fertilizer'],
 ['potato products'],
 ['sparkling wine'],
 ['dish cleaner'],
 ['male cosmetics'],
 ['meat spreads'],
 ['tidbits'],
 ['cleaner'],
 ['jam'],
 ['kitchen towels'],
 ['male

In [42]:
rules_df = pd.DataFrame(data={"Item": list_item, "Add Item": list_lifted_item, "Lift": list_lift})
rules_df

,Item,Add Item,Lift
0,[Instant food products],[cake bar],2.919850
1,[Instant food products],[canned fish],2.824638
2,[Instant food products],[chocolate],2.120574
3,[Instant food products],[condensed milk],2.794265
4,[Instant food products],[flower (seeds)],3.878607
...,...,...,...
468,[sugar],[spread cheese],2.166759
469,[spread cheese],[turkey],2.549712
470,[turkey],[spread cheese],2.549712
471,[vinegar],[white wine],2.266279


In [43]:
rules_df.sort_values("Lift").head(10)

,Item,Add Item,Lift
39,[sparkling wine],[beef],2.000280
147,[cocoa drinks],[fruit/vegetable juice],2.001027
59,[brandy],[specialty bar],2.001540
205,[misc. beverages],[detergent],2.001712
204,[detergent],[misc. beverages],2.001712
269,[photo/film],[frozen fish],2.004886
422,[photo/film],[seasonal products],2.004886
361,[liquor (appetizer)],[pot plants],2.006176
287,[jam],[frozen vegetables],2.006324
195,[dessert],[spread cheese],2.006480
